In [1]:
from business_models import hahn
import pandas as pd
import numpy as np



In [2]:
query = '''
    use hahn;
    PRAGMA AnsiInForEmptyOrNullableItemsCollections;

    SELECT *
    from `//home/taxi-delivery/analytics/dev/marketing_adhocs/cargo_churn_users`
    ;
        '''

In [3]:
df = hahn(query)

In [4]:
df.columns = ['phone_pd_id', 'deliveries']

In [7]:
id_to_remove = pd.read_csv('id_to_remove.csv')

In [19]:
id_to_remove.columns = ['phone_pd_id']
id_to_remove['x'] = [1 for i in range(len(id_to_remove['phone_pd_id']))]

In [22]:
df = df.merge(id_to_remove, how='outer', on='phone_pd_id')
df = df[df['x'].isna()]

In [24]:
df = df.sort_values(by='deliveries', ascending=False).head(30000)

In [144]:
def splitovalka(pdid, sault):
    pdid_s = pdid+sault
    hash_res = binascii.crc32(pdid_s.encode('utf8'))
    if hash_res%100<33:
        return 'C'
    elif hash_res%100<66:
        return 'B'
    else:
        return 'A'

In [145]:
df['exp_group'] = df['phone_pd_id'].apply(splitovalka, args=('first_iter',))

In [146]:
df

,phone_pd_id,deliveries,x,exp_group
125607,dcacf33b73ad4aaa85f867aeed44fbe8,199,NaN,A
132973,e99225acc33a4c33b7c29697c9dff5e9,199,NaN,C
130848,e5e4eb035c304eba9f3b0e235e35e1ad,198,NaN,C
99890,af1f6c3bf7bb443c8457490f593bd4fa,197,NaN,C
53449,5dbb8aea84624971a6a6732603e32897,196,NaN,C
...,...,...,...,...
19200,219f452cab8d43338a569b4b3eb1b6ce,6,NaN,B
20895,249b3c85fc5844aea77971cafe2d9982,6,NaN,A
19198,219e6c0ec155487b8443611d0668a226,6,NaN,B
50891,593ef909e6f8456983c282f7ecf32f8c,6,NaN,C


In [29]:
df[['phone_pd_id', 'deliveries', 'exp_group']].to_excel("churn_users_split.xlsx", sheet_name='first_iter', index=False)  